In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

from scipy.optimize import minimize

In [2]:
# pd.set_option('display.max_rows', 20)

In [22]:
# import yfinance as yf
# import pandas as pd
# import numpy as np
# from scipy import optimize
# 
# import os
# cwd = os.getcwd()
# print(cwd)
# os.chdir(r'C:\Users\mitch\Documents\QCF\Finance and Investments\StockTrack-Trading-Simulation')

C:\Users\mitch\Documents\QCF\Finance and Investments\StockTrack-Trading-Simulation


# Reusable functions

In [3]:
def calculate_return(df):
    if df.empty:
        return None
    # Get the closing price at the start of the year
    start_price = df['Close'].iloc[0]

    # Get the closing price at the end of the year
    end_price = df['Close'].iloc[-1]

    # Calculate the one-year return
    return (end_price - start_price) / start_price
# 
# 
# def MaximizeSharpeRatioOptmzn(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
#     # define maximization of Sharpe Ratio using principle of duality
#     def f(x, MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
#         funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T))
#         funcNumer = np.matmul(np.array(MeanReturns), x.T) - RiskFreeRate
#         func = -(funcNumer / funcDenomr)
#         return func
# 
#     #define equality constraint representing fully invested portfolio
#     def constraintEq(x):
#         A = np.ones(x.shape)
#         b = 1
#         constraintVal = np.matmul(A, x.T) - b
#         return constraintVal
# 
#     #define bounds and other parameters
#     xinit = np.repeat(0.33, PortfolioSize)
#     cons = ({'type': 'eq', 'fun': constraintEq})
#     lb = 0
#     ub = 1
#     bnds = tuple([(lb, ub) for x in xinit])
# 
#     #invoke minimize solver
#     opt = minimize(f, x0=xinit, args=(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize), method='SLSQP',
#                    bounds=bnds, constraints=cons, tol=10 ** -3)
# 
#     return opt


In [4]:
largest_market_cap = pd.read_excel('./Top Companies by Market Cap.xlsx', sheet_name='Sheet1')
largest_market_cap

,Symbol,Company Name,Industry,Market Cap
0,AAPL,Apple Inc.,Consumer Electronics,"2,811.03B"
1,MSFT,Microsoft Corporation,Software - Infrastructure,"2,469.80B"
2,GOOGL,Alphabet Inc.,Internet Content & Information,"1,779.01B"
3,GOOG,Alphabet Inc.,Internet Content & Information,"1,778.79B"
4,AMZN,"Amazon.com, Inc.",Internet Retail,"1,360.20B"
...,...,...,...,...
495,RBLX,Roblox Corporation,Electronic Gaming & Multimedia,19.59B
496,WBA,"Walgreens Boots Alliance, Inc.",Pharmaceutical Retailers,19.51B
497,TW,Tradeweb Markets Inc.,Capital Markets,19.51B
498,ZM,"Zoom Video Communications, Inc.",Software - Application,19.51B


In [5]:
largest_market_cap = largest_market_cap[~largest_market_cap['Symbol'].str.contains('\.')].reset_index(drop=True)
top = largest_market_cap.head(100)
top_symbols = top['Symbol']

In [6]:
# ticker = yf.Ticker('AAPL')
# stock_info = ticker.info
# print(stock_info.get('beta'))

In [7]:
one_year_data = {}
three_year_data = {}
five_year_data = {}
for symbol in top_symbols:
    ticker = yf.Ticker(symbol)
    one_year_data[symbol] = ticker.history(period="1y")
    three_year_data[symbol] = ticker.history(period="3y")
    five_year_data[symbol] = ticker.history(period="5y")

In [8]:
one_year_return_data = {}
three_year_return_data = {}
five_year_return_data = {}
for key, df in one_year_data.items():
    one_year_return_data[key] = calculate_return(one_year_data[key])
    three_year_return_data[key] = calculate_return(three_year_data[key])
    five_year_return_data[key] = calculate_return(five_year_data[key])

one_year_return_data

{'AAPL': 0.2917556234842266,
 'MSFT': 0.553459551488625,
 'GOOGL': 0.42876567487462375,
 'GOOG': 0.4427631416577035,
 'AMZN': 0.5449935006568656,
 'NVDA': 2.2367650204634884,
 'META': 2.07691604679862,
 'TSLA': 0.44090666314715554,
 'LLY': 0.6648753083181432,
 'V': 0.2295151965443692,
 'UNH': 0.059583592817676945,
 'TSM': 0.25958162378282035,
 'NVO': 0.8694792063963371,
 'WMT': 0.047081798486593884,
 'JPM': 0.1844568647005163,
 'XOM': -0.025733217602796645,
 'JNJ': -0.1184949046371724,
 'MA': 0.21746454587135602,
 'AVGO': 0.9570143507900121,
 'PG': 0.06324002389728373,
 'CVX': -0.17425568962349172,
 'ORCL': 0.48412320748082016,
 'HD': -0.010367514244657622,
 'ABBV': -0.08171345212007528,
 'MRK': -0.0016361924645046948,
 'COST': 0.12302466333147209,
 'ADBE': 0.8970108359285948,
 'TM': 0.29554725877407184,
 'ASML': 0.18916668012234183,
 'KO': -0.04048155620384998,
 'PEP': -0.06995223054590402,
 'BABA': 0.027167507209148054,
 'SHEL': 0.2638715317950637,
 'CSCO': 0.036063652093303276,
 'BA

In [9]:
def get_deciles(data):
    # Remove None values and sort the data
    filtered_data = {k: v for k, v in data.items() if v is not None}
    sorted_data = sorted(filtered_data.items(), key=lambda x: x[1])

    # Calculate the index for decile
    n = len(sorted_data)
    decile_index = n // 10

    # Get the top and bottom decile
    bottom_decile = dict(sorted_data[:decile_index])
    top_decile = dict(sorted_data[-decile_index:])

    return top_decile, bottom_decile


# Apply the function to each dataset
top_decile_one_year, bottom_decile_one_year = get_deciles(one_year_return_data)
top_decile_three_year, bottom_decile_three_year = get_deciles(three_year_return_data)
top_decile_five_year, bottom_decile_five_year = get_deciles(five_year_return_data)

# Find overlapping keys in the top deciles
overlapping_top_keys = set(top_decile_one_year.keys()) & set(top_decile_three_year.keys()) & set(
    top_decile_five_year.keys())

# Find overlapping keys in the bottom deciles
overlapping_bottom_keys = set(bottom_decile_one_year.keys()) & set(bottom_decile_three_year.keys())

# Print results
print("Top Decile Overlaps:", overlapping_top_keys)
print("Bottom Decile Overlaps:", overlapping_bottom_keys)


Top Decile Overlaps: {'NVO', 'AVGO', 'LLY', 'NVDA'}
Bottom Decile Overlaps: {'BMY', 'HDB', 'NEE'}


# Choosing top-50 and bottom-50 companies

In [10]:
'''''

target_long = pd.DataFrame(['ASML', 'GOOG', 'ACN', 'AAPL', 'SONY',
                            'INTU', 'V', 'MA', 'BX', 'TTE'],
                           columns=['Ticker'])
# TODO
target_short = pd.DataFrame([],
                            columns=['Ticker']
                            )
                            
                            '''''

"''\n\ntarget_long = pd.DataFrame(['ASML', 'GOOG', 'ACN', 'AAPL', 'SONY',\n                            'INTU', 'V', 'MA', 'BX', 'TTE'],\n                           columns=['Ticker'])\n# TODO\ntarget_short = pd.DataFrame([],\n                            columns=['Ticker']\n                            )\n                            \n                            "

In [11]:
# Get all unique keys in the top deciles
all_top_keys = set(top_decile_one_year.keys()) | set(top_decile_three_year.keys()) | set(top_decile_five_year.keys())

# Get all unique keys in the bottom deciles
all_bottom_keys = set(bottom_decile_one_year.keys()) | set(bottom_decile_three_year.keys()) | set(
    bottom_decile_five_year.keys())

all_top_keys

{'AAPL',
 'ADBE',
 'AMAT',
 'AMD',
 'ASML',
 'AVGO',
 'BP',
 'BX',
 'COP',
 'GE',
 'LLY',
 'META',
 'NFLX',
 'NOW',
 'NVDA',
 'NVO',
 'ORCL',
 'PDD',
 'SHEL',
 'TSLA',
 'XOM'}

In [12]:
all_bottom_keys

{'BA',
 'BABA',
 'BAC',
 'BMY',
 'BUD',
 'CRM',
 'CVX',
 'DIS',
 'HDB',
 'HON',
 'INTC',
 'JNJ',
 'NEE',
 'NKE',
 'PDD',
 'PFE',
 'RTX',
 'T',
 'UL',
 'UPS',
 'VZ',
 'WFC'}

In [13]:
target_long = pd.DataFrame(list(all_top_keys),
                           columns=['Ticker'])
# TODO
target_short = pd.DataFrame(list(all_bottom_keys),
                            columns=['Ticker'])


# Getting 2-month history

In [14]:
def get_history(target_df: pd.DataFrame, long=True) -> pd.DataFrame:
    history = pd.DataFrame(columns=list(target_long['Ticker']))
    for ticker in target_df['Ticker']:
        # TODO think about the appropriate price column to use
        history[ticker] = yf.Ticker(ticker).history(interval='5m')['High' if long else 'Low']
    return history

In [15]:
history_long = get_history(target_long)
history_long.info()
history_long.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1672 entries, 2023-10-23 09:30:00-04:00 to 2023-11-21 12:15:00-05:00
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NFLX    1672 non-null   float64
 1   SHEL    1672 non-null   float64
 2   NVO     1672 non-null   float64
 3   PDD     1672 non-null   float64
 4   LLY     1672 non-null   float64
 5   GE      1672 non-null   float64
 6   AMAT    1672 non-null   float64
 7   AMD     1672 non-null   float64
 8   ASML    1670 non-null   float64
 9   NOW     1672 non-null   float64
 10  NVDA    1672 non-null   float64
 11  META    1672 non-null   float64
 12  BX      1672 non-null   float64
 13  AAPL    1672 non-null   float64
 14  TSLA    1672 non-null   float64
 15  ADBE    1672 non-null   float64
 16  COP     1672 non-null   float64
 17  ORCL    1672 non-null   float64
 18  AVGO    1672 non-null   float64
 19  BP      1672 non-null   float64
 20  XOM     1672 non-null   fl

,NFLX,SHEL,NVO,PDD,LLY,GE,AMAT,AMD,ASML,NOW,...,META,BX,AAPL,TSLA,ADBE,COP,ORCL,AVGO,BP,XOM
Datetime,,,,,,,,,,,,,,,,,,,,,
2023-10-23 09:30:00-04:00,407.109985,67.690002,97.139999,103.190002,586.059998,107.440002,133.985001,101.470001,579.190002,540.960022,...,310.649994,94.300003,171.679993,210.479996,537.929993,123.160004,102.919998,858.765198,39.200001,110.959999
2023-10-23 09:35:00-04:00,407.509888,67.470001,96.720001,104.129898,580.919983,107.989998,133.059998,100.380096,577.005005,536.000000,...,310.200012,93.230003,171.054993,206.729996,534.340027,122.400002,103.019997,847.530029,39.009998,110.769997
2023-10-23 09:40:00-04:00,404.700012,67.300003,96.599998,103.889999,580.359985,107.959999,132.839905,100.099998,575.349976,532.080017,...,309.380005,93.059998,170.570007,205.250000,533.044983,122.720001,102.910004,845.780029,38.880501,110.669998
2023-10-23 09:45:00-04:00,406.480011,67.209999,96.800003,102.772202,579.250000,107.680000,133.890594,100.294998,580.280029,537.859985,...,309.970001,93.635002,171.100006,205.500000,535.726318,122.180000,102.919998,851.630005,38.750000,109.875000
2023-10-23 09:50:00-04:00,405.763611,67.260002,96.860001,103.000000,578.989990,107.769997,134.100006,100.870003,581.140015,538.429993,...,310.649994,93.720001,171.350006,206.990005,537.789978,122.459999,102.699997,852.200012,38.790001,110.269997


In [16]:
history_short = get_history(target_short)
history_short.dropna(axis=1, inplace=True)
history_short.info()
history_short.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1672 entries, 2023-10-23 09:30:00-04:00 to 2023-11-21 12:15:00-05:00
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PDD     1672 non-null   float64
 1   PFE     1672 non-null   float64
 2   WFC     1672 non-null   float64
 3   RTX     1672 non-null   float64
 4   BAC     1672 non-null   float64
 5   HDB     1672 non-null   float64
 6   UPS     1672 non-null   float64
 7   INTC    1672 non-null   float64
 8   DIS     1672 non-null   float64
 9   HON     1672 non-null   float64
 10  BMY     1672 non-null   float64
 11  JNJ     1672 non-null   float64
 12  T       1672 non-null   float64
 13  CVX     1672 non-null   float64
 14  VZ      1672 non-null   float64
 15  NEE     1672 non-null   float64
 16  BA      1672 non-null   float64
 17  CRM     1672 non-null   float64
 18  BUD     1672 non-null   float64
 19  BABA    1672 non-null   float64
 20  NKE     1672 non-null   fl

,PDD,PFE,WFC,RTX,BAC,HDB,UPS,INTC,DIS,HON,...,T,CVX,VZ,NEE,BA,CRM,BUD,BABA,NKE,UL
Datetime,,,,,,,,,,,,,,,,,,,,,
2023-10-23 09:30:00-04:00,103.190002,30.870001,40.365002,72.860001,26.150000,57.122101,151.460007,34.764999,82.320000,181.869904,...,15.3800,164.429993,31.490000,51.560001,180.119995,200.446594,52.660000,80.050003,102.349998,48.430000
2023-10-23 09:35:00-04:00,104.129898,31.115000,40.380001,72.860001,26.150000,56.970001,150.830002,34.750000,81.809998,181.800003,...,15.2900,162.539993,31.450001,51.540001,180.500000,199.580002,52.549999,80.004997,102.309998,48.419998
2023-10-23 09:40:00-04:00,103.889999,31.160000,40.139999,72.912003,26.040001,56.849998,150.889999,34.495899,81.510002,181.639999,...,15.3100,162.970001,31.440001,51.480000,179.509995,198.990005,52.528801,79.760002,102.070000,48.391499
2023-10-23 09:45:00-04:00,102.772202,31.389999,40.165001,72.970001,26.000000,56.970001,151.240005,34.630001,81.510002,182.169998,...,15.3300,162.860001,31.485001,51.560001,180.080002,199.800003,52.619999,79.730003,102.525002,48.450001
2023-10-23 09:50:00-04:00,103.000000,31.260000,40.189999,73.190002,26.010000,57.046101,151.580002,34.689999,81.799698,182.479996,...,15.3199,163.029999,31.549999,51.660000,180.389999,200.550003,52.735001,79.769997,102.599998,48.430000


# Getting 2-month compound returns

In [17]:
def get_returns(history_df: pd.DataFrame) -> np.ndarray:
    n = len(history_df.columns)
    returns = np.zeros(n, dtype=float)
    for i in range(n):
        # TODO think about the appropriate return calculation
        helper = history_df.iloc[:, i] / history_df.iloc[:, i].shift(1)
        helper = helper.iloc[1:]
        returns[i] = np.prod(helper) - 1
    return returns

In [18]:
returns_long = get_returns(history_long)
print(f'Top companies\' returns: {returns_long}')

Top companies' returns: [ 0.17123112 -0.01817112  0.06423716  0.13635042  0.01614173  0.11476167
  0.11236327  0.1694097   0.1802308   0.23107068  0.19627769  0.08911094
  0.12311771  0.10813725  0.1502281   0.1341067  -0.06536215  0.13408476
  0.14899855 -0.08890308 -0.05776853]


In [19]:
returns_short = get_returns(history_short)
print(f'Bottom companies\' returns: {returns_short}')

Bottom companies' returns: [ 0.1365442  -0.02380954  0.05586516  0.07891847  0.13804974  0.03488844
 -0.01135614  0.25542935  0.14753397  0.05146587 -0.13468011 -0.01465291
  0.05201562 -0.1241257   0.18005716  0.1216059   0.21141463  0.12359106
  0.17699015 -0.01255471  0.03556423 -0.01858355]


# Getting weights

In [20]:
def sharpe_long(weights):
    return -np.dot(returns_long, weights) / np.sqrt(weights @ history_long.cov().to_numpy() @ weights)


def sharpe_short(weights):
    return -np.dot(returns_short, weights) / np.sqrt(weights @ history_short.cov().to_numpy() @ weights)

In [49]:
'''''
def get_weights(history_df: pd.DataFrame, long=True) -> np.ndarray:
    n = len(history_df.columns)
    weights = np.ones(n)

    bounds = [(0.0, None)] * n if long else [(None, 0.0)] * n
    constraints = ({'type': 'eq', 'fun': lambda weights: weights.sum() - 1.0})
    optimal_weights = minimize(sharpe_long if long else sharpe_short,
                               weights, 
                               method='SLSQP',
                               constraints=constraints,
                               bounds=bounds
                               ).x

    return optimal_weights
    '''''

"''\ndef get_weights(history_df: pd.DataFrame, long=True) -> np.ndarray:\n    n = len(history_df.columns)\n    weights = np.ones(n)\n\n    bounds = [(0.0, None)] * n if long else [(None, 0.0)] * n\n    constraints = ({'type': 'eq', 'fun': lambda weights: weights.sum() - 1.0})\n    optimal_weights = minimize(sharpe_long if long else sharpe_short,\n                               weights, \n                               method='SLSQP',\n                               constraints=constraints,\n                               bounds=bounds\n                               ).x\n\n    return optimal_weights\n    "

In [34]:
def get_weights(history_df: pd.DataFrame, long=True) -> dict:
    n = len(history_df.columns)
    weights = np.ones(n) if long else np.full(n, -1)

    bounds = [(0.0, 0.10)] * n if long else [(None, 0.0)] * n
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}) if long else \
        ({'type': 'eq', 'fun': lambda w: np.sum(w) + 1.0})
    optimal_weights = minimize(sharpe_long if long else sharpe_short,
                               weights,
                               method='SLSQP',
                               constraints=constraints,
                               bounds=bounds
                               ).x

    # Create a dictionary with column names as keys and weights as values
    weights_dict = dict(zip(history_df.columns, optimal_weights))
    return weights_dict

In [35]:
weights_long = get_weights(history_long, True)
weights_long_vw = pd.DataFrame(list(weights_long.items()), columns=['Stock', 'Weight'])

weights_long_vw['Weight'] = weights_long_vw['Weight'].apply(lambda x: f"{x * 100:.2f}%")

#print(f'Optimal weights for top companies: {weights_long}')

#print(f'Optimal Sharpe ratio: {-sharpe_long(weights_long)}')

#history_long.columns

print(weights_long_vw)



   Stock  Weight
0   NFLX   0.00%
1   SHEL  10.00%
2    NVO  10.00%
3    PDD  10.00%
4    LLY   0.00%
5     GE  10.00%
6   AMAT   4.52%
7    AMD  10.00%
8   ASML   0.00%
9    NOW   0.00%
10  NVDA   0.00%
11  META   0.00%
12    BX  10.00%
13  AAPL   5.48%
14  TSLA   0.00%
15  ADBE   0.00%
16   COP  10.00%
17  ORCL  10.00%
18  AVGO   0.00%
19    BP   0.00%
20   XOM  10.00%


In [36]:
weights_short = get_weights(history_short, False)
weights_short_vw = pd.DataFrame(list(weights_short.items()), columns=['Stock', 'Weight'])
weights_short_vw['Weight'] = weights_short_vw['Weight'].apply(lambda x: f"{x * 100:.2f}%")

#print(f'Optimal weights for top companies: {weights_long}')

#print(f'Optimal Sharpe ratio: {-sharpe_long(weights_long)}')

#history_long.columns

print(weights_short_vw)

# print(f'Optimal weights for bottom companies: {weights_short}')
# print(f'Optimal Sharpe ratio: {-sharpe_short(np.array(list(weights_short.values())))}')

   Stock   Weight
0    PDD    0.00%
1    PFE  -15.15%
2    WFC   -0.00%
3    RTX    0.00%
4    BAC    0.00%
5    HDB   -0.00%
6    UPS    0.00%
7   INTC    0.00%
8    DIS   -0.00%
9    HON    0.00%
10   BMY  -76.63%
11   JNJ    0.00%
12     T    0.00%
13   CVX    0.00%
14    VZ    0.00%
15   NEE    0.00%
16    BA   -6.44%
17   CRM   -0.73%
18   BUD   -0.00%
19  BABA   -1.05%
20   NKE    0.00%
21    UL   -0.00%
